In [1]:
!pip install scikit-learn==1.5.0 

In [2]:
pip freeze | grep "scikit-learn==1.5.0"

scikit-learn==1.5.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -V

Python 3.12.3


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
##What's the standard deviation of the predicted duration for this dataset?

In [10]:
round(y_pred.std(), 3)

np.float64(6.247)

In [11]:
## Preparing the output as per question2 

In [12]:
df['ride_id'] = (
    df['tpep_pickup_datetime'].dt.year.astype(str).str.zfill(4) + '/' +
    df['tpep_pickup_datetime'].dt.month.astype(str).str.zfill(2) + '_' +
    df.index.astype(str)
)

df['ride_id'].head()

0    2023/03_0
1    2023/03_1
2    2023/03_2
3    2023/03_3
4    2023/03_4
Name: ride_id, dtype: object

In [13]:
df_result = df[["ride_id"]].copy()
df_result['duration'] = y_pred

output_file = "output/result.parquet"
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [15]:
import os

size_bytes = os.path.getsize(output_file)
size_mb = size_bytes / (1024 * 1024)

print(f"File size: {size_bytes:,} bytes ({size_mb:.2f} MB)")

File size: 68,640,918 bytes (65.46 MB)


In [16]:
## Question3 - Creating the scoring script

In [17]:
!jupyter nbconvert --to script starter-Q1.ipynb

[NbConvertApp] Converting notebook starter-Q1.ipynb to script
[NbConvertApp] Writing 3090 bytes to starter-Q1.py


In [18]:
## Question 4- Now let's put everything into a virtual environment. We'll use pipenv for that.

In [19]:
!pwd

/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/MLOps Zoompcamp/Models Deployment


In [20]:
!export PIPENV_PIPFILE="/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/MLOps Zoompcamp/Models Deployment/Pipfile"

In [21]:
!pipenv install scikit-learn==1.5.0 pyarrow pandas flask google-cloud-storage prefect --python=3.12

Courtesy Notice:
Pipenv found itself running within a virtual environment,  so it will 
automatically use that environment, instead of  creating its own for any 
project. You can set
PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and 
create  its own instead.
You can set PIPENV_VERBOSITY=-1 to suppress this warning.
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing scikit-learn==1.5.0...
✔ Installation Succeeded
Installing pyarrow...
✔ Installation Succeeded
Installing pandas...
✔ Installation Succeeded
Installing flask...
✔ Installation Succeeded
Installing google-cloud-storage...
✔ Installation Succeeded
Installing prefect...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (f89251)...
All dependencies are now up-to-date!
Upgrading sci

In [23]:
from json import load
with open("Pipfile.lock", "rb") as file_in:
    pipfile = load(file_in)
    print(f'hash for the Scikit-Learn dependency is :{pipfile["default"]["scikit-learn"]["hashes"][0]}')

hash for the Scikit-Learn dependency is :sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c


In [24]:
## question5 - Parametrize the script
# Let's now make the script configurable via CLI. We'll create two parameters: year and month.
# Run the script for April 2023.

In [26]:
import os
os.chdir("/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/MLOps Zoompcamp/Models Deployment")

In [27]:
!pwd

/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/MLOps Zoompcamp/Models Deployment


In [38]:
!python starter-Q1.py 2023 04

11:41:24.165 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8978
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.
Traceback (most recent call last):
  File "/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/MLOps Zoompcamp/Models Deployment/starter-Q1.py", line 103, in <module>
    taxi_pipeline(project_id, bucket_name, year, month)
  File "/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/lib/python3.12/site-packages/prefect/flows.py", line 1702, in __call__
    return run_flow(
           ^^^^^^^^^
  File "/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/lib/python3.12/site-packages/prefect/flow_engine.py", line 1552, in run_flow
    ret_val = run_flow_sync(**kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/c/Users/shubham_yadav/Documents/COXA-ENIT/lab/venv/lib/python3.12/site-packages/prefect/flow_engine.py", line 1392, in run_flow_sync
    with engine.start():
 

In [43]:
!docker build -t ride-duration-prediction:v1 .

failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  68.88MB
Step 1/11 : FROM agrigorev/zoomcamp-model:mlops-2024-3.10.13-slim
 ---> 13e5353db264
Step 2/11 : RUN apt-get update && apt-get install -y gcc && apt-get clean && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> dab9ebdcb8d8
Step 3/11 : RUN pip install -U pip
 ---> Using cache
 ---> 1ade24a81bb4
Step 4/11 : RUN pip install pipenv
 ---> Using cache
 ---> a2328c7dcef4
Step 5/11 : WORKDIR /app
 ---> Using cache
 ---> 0c448f02dd48
Step 6/11 : COPY ["Pipfile", "Pipfile.lock", "./"]
 ---> Using cache
 ---> 4e02c9f8fc5d
Step 7/11 : RUN pipenv install --system --deploy
 ---> Using cache
 ---> 611a423ac76c
Step 8/11 : RUN pip install

In [44]:
!docker run --rm ride-duration-prediction:v1

11:55:28.811 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8409
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.
11:55:36.142 | INFO    | Flow run 'lemon-lobster' - Beginning flow run 'lemon-lobster' for flow 'Taxi ML Pipeline'
11:55:56.247 | INFO    | Task run 'load_data-c0e' - Finished in state Completed()
11:55:58.963 | INFO    | Task run 'transform_data-a20' - Finished in state Completed()
11:56:12.469 | INFO    | Task run 'prepare_data-9de' - Finished in state Completed()
11:56:12.664 | INFO    | Task run 'apply_model-393' - Finished in state Completed()
11:56:12.766 | INFO    | Flow run 'lemon-lobster' - Finished in state Completed()
The mean predicted duration is 0.192 minutes
11:56:12.775 | INFO    | prefect - Stopping temporary server on http://127.0.0.1:8409
